In [ ]:
# Install required libraries
!pip install sentence-transformers
!pip install langchain


In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

cv_text = """Name: Md Shafiqul Islam
Degree: BSc in Electrical and Electronic Engineering
Skills: Python, SQL, Power BI, Data Analysis, AI, etc.
Work Experience: Assistant Project Manager at Floating Solar Power Plant
Industry Experience: Renewable energy
Hobbies: Reading, Travel, Technology"""

# ✅ Fine-tuned splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=60,  # Even smaller chunks for precise retrieval
    chunk_overlap=30,  # More overlap to keep context
    separators=["\n\n", "\n", ". ", ", "]  # Logical breakpoints
)

cv_chunks = text_splitter.split_text(cv_text)

# Print the refined chunks
print("CV Chunks:", cv_chunks)


CV Chunks: ['Name: Md Shafiqul Islam', 'Degree: BSc in Electrical and Electronic Engineering', 'Skills: Python, SQL, Power BI, Data Analysis, AI, etc.', '\nWork Experience: Assistant Project Manager at Floating Solar Power Plant', 'Industry Experience: Renewable energy', 'Hobbies: Reading, Travel, Technology']


In [19]:
# Step 3: Embed CV Chunks using Sentence Transformers
from sentence_transformers import SentenceTransformer, util

# Load pre-trained model for embeddings
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [20]:
# Create embeddings for CV chunks
cv_embeddings = model.encode(cv_chunks, convert_to_tensor=True)

# Step 4: Define the Question Answering Function Using Semantic Search
def vector_search_answer(question, cv_chunks, cv_embeddings):
    # Embed the question
    question_embedding = model.encode([question], convert_to_tensor=True)

    # Calculate cosine similarity between question and CV embeddings
    similarities = util.pytorch_cos_sim(question_embedding, cv_embeddings)[0]

    # Get the index of the most similar CV chunk
    best_match_idx = similarities.argmax().item()

    # Return the most relevant chunk
    return cv_chunks[best_match_idx]

In [32]:
# Step 5: Test the QA system with a sample question
question = "What are your skills?"
answer = vector_search_answer(question, cv_chunks, cv_embeddings)
print(f"Answer to the question '{question}':", answer)


Answer to the question 'What are your skills?': Skills: Python, SQL, Power BI, Data Analysis, AI, etc.


In [31]:
def cv_assistant():
    print("CV Assistant is ready. Ask me a question related to your CV (type 'exit' to quit):")
    while True:
        question = input("Ask your question: ")
        if question.lower() == 'exit':
            print("Goodbye!")
            break
        answer = vector_search_answer(question, cv_chunks, cv_embeddings)
        print(f"Answer: {answer}")